In [1]:
import json
import pickle
import os
import datetime
import skimage.io
from skimage import measure
import numpy as np
import matplotlib.pyplot as plt
import pycocotools
import pycocotools.mask

In [4]:
# True if only want one class (hand), False if want to separate left and right hand
one_hand = True
# Directory to RHD dataset
dataset_dir = os.path.join(os.getcwd(), '../datasets/RHD_published_v2')
# 'train' or 'val'
subset = 'val'
# Only use 2017
year = 2017

if subset == 'train':
    annotations_file = os.path.join(dataset_dir, 'training/anno_training.pickle')
else:
    annotations_file = os.path.join(dataset_dir, 'evaluation/anno_evaluation.pickle')

In [5]:
with open(annotations_file, 'rb') as f:
    anno_all = pickle.load(f)
    
anno_all[0].keys()

dict_keys(['K', 'xyz', 'uv_vis'])

In [6]:
info = {'description': 'Rendered Handpose Dataset',
        'url': 'https://lmb.informatik.uni-freiburg.de/projects/hand3d/',
        'version': '1.1',
        'year': 2017,
        'contributor': 'Christian Zimmermann and Thomas Brox',
        'date_created': datetime.datetime.utcnow().isoformat(' ')}

licenses = [{'url': 'https://lmb.informatik.uni-freiburg.de/projects/hand3d/',
            'id': 1,
            'name': 'Rendered Handpose Dataset License'}]

if one_hand:
    categories = [{'supercategory': 'hand', 'name': 'hand', 'id': 1, 
                   'keypoints': ['wrist',

                                 'thumb_tip',
                                 'thumb_upper_joint',
                                 'thumb_lower_joint',
                                 'thumb_palm',

                                 'index_tip',
                                 'index_upper_joint',
                                 'index_lower_joint',
                                 'index_palm',

                                 'middle_tip',
                                 'middle_upper_joint',
                                 'middle_lower_joint',
                                 'middle_palm',

                                 'ring_tip',
                                 'ring_upper_joint',
                                 'ring_lower_joint',
                                 'ring_palm',

                                 'pinky_tip',
                                 'pinky_upper_joint',
                                 'pinky_lower_joint',
                                 'pinky_palm'], 
                   'skeleton' : [[1, 5],
                                 [5, 4],
                                 [4, 3],
                                 [3, 2],

                                 [1, 9],
                                 [9, 8],
                                 [8, 7],
                                 [7, 6],

                                 [1, 13],
                                 [13, 12],
                                 [12, 11],
                                 [11, 10],

                                 [1, 17],
                                 [17, 16],
                                 [16, 15],
                                 [15, 14],

                                 [1, 21],
                                 [21, 20],
                                 [20, 19],
                                 [19, 18]]}
                 ]
else:
    categories = [{'supercategory': 'hand', 'name': 'left', 'id': 1, 
                   'keypoints': ['wrist',

                                 'thumb_tip',
                                 'thumb_upper_joint',
                                 'thumb_lower_joint',
                                 'thumb_palm',

                                 'index_tip',
                                 'index_upper_joint',
                                 'index_lower_joint',
                                 'index_palm',

                                 'middle_tip',
                                 'middle_upper_joint',
                                 'middle_lower_joint',
                                 'middle_palm',

                                 'ring_tip',
                                 'ring_upper_joint',
                                 'ring_lower_joint',
                                 'ring_palm',

                                 'pinky_tip',
                                 'pinky_upper_joint',
                                 'pinky_lower_joint',
                                 'pinky_palm'], 
                   'skeleton' : [[1, 5],
                                 [5, 4],
                                 [4, 3],
                                 [3, 2],

                                 [1, 9],
                                 [9, 8],
                                 [8, 7],
                                 [7, 6],

                                 [1, 13],
                                 [13, 12],
                                 [12, 11],
                                 [11, 10],

                                 [1, 17],
                                 [17, 16],
                                 [16, 15],
                                 [15, 14],

                                 [1, 21],
                                 [21, 20],
                                 [20, 19],
                                 [19, 18]]},
                  {'supercategory': 'hand', 'name': 'right', 'id': 2, 
                   'keypoints': ['wrist',

                                 'thumb_tip',
                                 'thumb_upper_joint',
                                 'thumb_lower_joint',
                                 'thumb_palm',

                                 'index_tip',
                                 'index_upper_joint',
                                 'index_lower_joint',
                                 'index_palm',

                                 'middle_tip',
                                 'middle_upper_joint',
                                 'middle_lower_joint',
                                 'middle_palm',

                                 'ring_tip',
                                 'ring_upper_joint',
                                 'ring_lower_joint',
                                 'ring_palm',

                                 'pinky_tip',
                                 'pinky_upper_joint',
                                 'pinky_lower_joint',
                                 'pinky_palm'], 
                   'skeleton' : [[1, 5],
                                 [5, 4],
                                 [4, 3],
                                 [3, 2],

                                 [1, 9],
                                 [9, 8],
                                 [8, 7],
                                 [7, 6],

                                 [1, 13],
                                 [13, 12],
                                 [12, 11],
                                 [11, 10],

                                 [1, 17],
                                 [17, 16],
                                 [16, 15],
                                 [15, 14],

                                 [1, 21],
                                 [21, 20],
                                 [20, 19],
                                 [19, 18]]}
                 ]

In [7]:
def create_image_info(image_id, file_name, image_size, 
                      date_captured=datetime.datetime.utcnow().isoformat(' '),
                      license_id=1, coco_url="", flickr_url=""):

    image_info = {
            "id": image_id,
            "file_name": file_name,
            "width": image_size[0],
            "height": image_size[1],
            "date_captured": date_captured,
            "license": license_id,
            "coco_url": coco_url,
            "flickr_url": flickr_url
    }

    return image_info

In [8]:
def create_annotation_info(annotation_id, image_id, category_id, binary_mask, keypoints, 
                           image_size=None, tolerance=2, is_crowd=0, bounding_box=None):

    if image_size is not None:
        binary_mask = resize_binary_mask(binary_mask, image_size)

    binary_mask_encoded = pycocotools.mask.encode(np.asfortranarray(binary_mask.astype(np.uint8)))
    
    num_keypoints = int(np.sum(keypoints[:,2]))
    
    kp = keypoints.copy()
    kp[np.where(kp[:,2] == 1), 2] = 2
    kp[np.where((((kp[:,0] <= 320) & (kp[:,0] >=0)) & ((kp[:,1] <= 320) & (kp[:,1] >=0))) & (kp[:,2] != 2)), 2] = 1
    kp[np.where(kp[:,2] == 0), :2] = 0

    area = pycocotools.mask.area(binary_mask_encoded)
    if area < 1:
        return None
    
    if bounding_box is None:
        bounding_box = pycocotools.mask.toBbox(binary_mask_encoded)

    if is_crowd == 1:
        segmentation = binary_mask_to_rle(binary_mask)
    else :
        segmentation = binary_mask_to_polygon(binary_mask, tolerance)
        if not segmentation:
            return None

    annotation_info = {
        "id": annotation_id,
        "image_id": image_id,
        "category_id": category_id,
        "iscrowd": is_crowd,
        "area": area.tolist(),
        "bbox": bounding_box.tolist(),
        "segmentation": segmentation,
        "keypoints": kp.flatten().tolist(),
        "num_keypoints": num_keypoints, 
        "width": binary_mask.shape[1],
        "height": binary_mask.shape[0],
    } 

    return annotation_info

In [9]:
def resize_binary_mask(array, new_size):
    image = Image.fromarray(array.astype(np.uint8)*255)
    image = image.resize(new_size)
    return np.asarray(image).astype(np.bool_)

def close_contour(contour):
    if not np.array_equal(contour[0], contour[-1]):
        contour = np.vstack((contour, contour[0]))
    return contour

def binary_mask_to_rle(binary_mask):
    rle = {'counts': [], 'size': list(binary_mask.shape)}
    counts = rle.get('counts')
    for i, (value, elements) in enumerate(groupby(binary_mask.ravel(order='F'))):
        if i == 0 and value == 1:
                counts.append(0)
        counts.append(len(list(elements)))

    return rle

def binary_mask_to_polygon(binary_mask, tolerance=0):
    """Converts a binary mask to COCO polygon representation
    Args:
        binary_mask: a 2D binary numpy array where '1's represent the object
        tolerance: Maximum distance from original points of polygon to approximated
            polygonal chain. If tolerance is 0, the original coordinate array is returned.
    """
    polygons = []
    # pad mask to close contours of shapes which start and end at an edge
    padded_binary_mask = np.pad(binary_mask, pad_width=1, mode='constant', constant_values=0)
    contours = measure.find_contours(padded_binary_mask, 0.5)
    contours = np.subtract(contours, 1)
    for contour in contours:
        contour = close_contour(contour)
        contour = measure.approximate_polygon(contour, tolerance)
        if len(contour) < 3:
            continue
        contour = np.flip(contour, axis=1)
        segmentation = contour.ravel().tolist()
        # after padding and subtracting 1 we may get -0.5 points in our segmentation 
        segmentation = [0 if i < 0 else i for i in segmentation]
        polygons.append(segmentation)

    return polygons

In [11]:
images = []
annotations = []

annotation_id = 0
hand_id = 1
left_id = 1
right_id = 2

for sample_id, anno in anno_all.items():
    image_id = '%.5d' % sample_id
    file_name = 'color/%.5d.png' % sample_id
    image_size = (320, 320)
    
    image_info = create_image_info(image_id, file_name, image_size)
    
    images.append(image_info)

    if subset == 'train':
        mask = skimage.io.imread(os.path.join(dataset_dir, 'training', 'mask', '%.5d.png' % sample_id))
    else:
        mask = skimage.io.imread(os.path.join(dataset_dir, 'evaluation', 'mask', '%.5d.png' % sample_id))
    
    left_mask = (mask > 1) & (mask < 18)
    right_mask = mask > 17
    
    left_keypoints = anno['uv_vis'][:21]
    right_keypoints = anno['uv_vis'][21:]
    
    if one_hand == 'one_hand':
        left_anno = create_annotation_info('%.6d' % annotation_id, image_id, hand_id, left_mask, left_keypoints)
    else:
        left_anno = create_annotation_info('%.6d' % annotation_id, image_id, left_id, left_mask, left_keypoints)
        
    if left_anno is not None:
        annotations.append(left_anno)
        annotation_id += 1
    
    if one_hand == 'one_hand':
        right_anno = create_annotation_info('%.6d' % annotation_id, image_id, hand_id, right_mask, right_keypoints)
    else:
        right_anno = create_annotation_info('%.6d' % annotation_id, image_id, right_id, right_mask, right_keypoints)
    
    if right_anno is not None:
        annotations.append(right_anno)
        annotation_id += 1

In [12]:
custom = {'info': info,
          'licenses': licenses,
          'images': images,
          'annotations': annotations,
          'categories': categories}

In [17]:
if not os.path.exists(os.path.join(dataset_dir, 'annotations')):
    os.mkdir(os.path.join(dataset_dir, 'annotations'))

with open(os.path.join(dataset_dir, 'annotations/instances_{}{}.json'.format(subset, year)), 'w') as f:
    json.dump(custom, f)

if subset == 'train':
    os.rename(os.path.join(dataset_dir, 'training'), os.path.join(dataset_dir, '{}{}'.format(subset, year)))
else:
    os.rename(os.path.join(dataset_dir, 'evaluation'), os.path.join(dataset_dir, '{}{}'.format(subset, year)))